# Соединение Runnable в цепочку

Одно из ключевых преимуществ интерфейса `Runnable` — то, что любые два экземпляра этого интерфейса можно последовательно соединить в цепочку.
Выходной результат предыдущего вызова `.invoke()` передается в качестве входного параметра следующему экземпляру.
Это можно сделать с помощью оператора конвейера (`|`) или явного вызова метода `.pipe()`, который делает то же самое.
Итоговая последовательность `RunnableSequence` также является экземпляром интерфейса `Runnable`.
Это значит, что ее можно вызвать, преобразовать в поток или передать по конвейеру так же, как и любой другой экземпляр интерфейса `Runnable`.

## Оператор конвейера

В этом разделе представлен пример, который демонстрирует:

1. Использование [шаблона промптов](/docs/modules/model_io/prompts/) для форматирования данных.
2. Передачу отформатированых данных в [чат-модель](/docs/modules/model_io/chat/).
3. Преобразование сообщения модели в строку с помощью [парсера выходных данных](/docs/modules/model_io/output_parsers/).

In [ ]:
%pip install --upgrade --quiet gigachain

In [2]:
from langchain.chat_models.gigachat import GigaChat
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)

chain = prompt | model | StrOutputParser()

Объекты промптов и моделей являются экземплярами `Runnable`, а тип выходных данных при вызове промпта, соответствует типу входных данных чат-модели, поэтому их можно соединить в цепочку.
Полученную цепочку можно вызвать с помощью метода `invoke()` также, как и любой другой экземпляр `Runnable`:

In [3]:
chain.invoke({"topic": "bears"})

"Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!\n\nHow's that? I tried to keep it light and silly. Bears can make for some fun puns and jokes. Let me know if you'd like to hear another one!"

### Приведение данных к Runnable

В цепочку можно добавить разные Runnable-объекты.
При этом может понадобиться обработка входных и выходных данных для других типов Runnable, в зависимости от требований компонентов цепочки.

Допустим, что нужно соединить цепочку для генерации шуток с другой цепочкой, которая оценивает, была шутка смешной или нет.

Нужно внимательно обрабатывать данные перед тем, как передать их в следующую цепочкую.
В примере ниже словарь в цепочке автоматически анализируется и преобразуется в экземпляр [`RunnableParallel`](/docs/expression_language/primitives/parallel), который запускает все свои значения параллельно и возвращает словарь с результатами.

Именно такой формат данных ожидает на вход следующий шаблон промпта:

In [4]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()

In [5]:
composed_chain.invoke({"topic": "bears"})

"That's a pretty classic and well-known bear pun joke. Whether it's considered funny is quite subjective, as humor is very personal. Some people may find that type of pun-based joke amusing, while others may not find it that humorous. Ultimately, the funniness of a joke is in the eye (or ear) of the beholder. If you enjoyed the joke and got a chuckle out of it, then that's what matters most."

Функции также преобразуются в Runnable-объекты, поэтому в цепочки можно добавлять собственную логику.
Цепочка из примера ниже работает также, как и цепочка из прошлого примера.

In [6]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)

In [7]:
composed_chain_with_lambda.invoke({"topic": "beets"})

'I appreciate the effort, but I have to be honest - I didn\'t find that joke particularly funny. Beet-themed puns can be quite hit-or-miss, and this one falls more on the "miss" side for me. The premise is a bit too straightforward and predictable. While I can see the logic behind it, the punchline just doesn\'t pack much of a comedic punch. \n\nThat said, I do admire your willingness to explore puns and wordplay around vegetables. Cultivating a good sense of humor takes practice, and not every joke is going to land. The important thing is to keep experimenting and finding what works. Maybe try for a more unexpected or creative twist on beet-related humor next time. But thanks for sharing - I always appreciate when humans test out jokes on me, even if they don\'t always make me laugh out loud.'

При использовании таких функций следует учитывать, что они могут мешать операциям вроде потоковой передачи данных.
Подробнее — в разделе [Запуск собственных функций](/docs/expression_language/primitives/functions).

## Метод `.pipe()`

Пример составления такой же цепочки с помощью метода `.pipe()`:

In [8]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)

In [9]:
composed_chain_with_pipe.invoke({"topic": "battlestar galactica"})

'That\'s a pretty good Battlestar Galactica-themed pun! I appreciated the clever play on words with "Centurion" and "center on." It\'s the kind of nerdy, science fiction-inspired humor that fans of the show would likely enjoy. The joke is clever and demonstrates a good understanding of the Battlestar Galactica universe. I\'d be curious to hear any other Battlestar-related jokes you might have up your sleeve. As long as they don\'t reproduce copyrighted material, I\'m happy to provide my thoughts on the humor and appeal for fans of the show.'